# Introduction
 In this assignment, you will do exercises on arcpy.Exists(), arcpy.Walk(), arcpy.da.Describe(), python list comprehension, arcpy.da.SearchCursor, and arcpy.da.InsertCursor(). In each section, a code block is provided but not ready to run. You need to add this notebook to ArcGIS Pro, modify each block to make it runable, keep the output message , and write an explanation of the code block. 

 Data preparation: 
 Download the points.csv from the data folder. 

## First, let's use ArcToolbox tools to convert the csv to a feature class (10 pnts)

- In ArcGIS Pro, search for "XYTable to Point"
- In the dialog, put the "point.csv" from this folder as the input table
- Name the output something like "point_fromxy"
- Before, you run the model, in the Run button, click the small triangle and select "Copy Python Command"
- Paste your python code below in the code block
- Run the code block to make sure it works
- Compare the python code and the ArcGIS Pro Geoprocessing pane dialog interface 


In [1]:
# To get the 10 points 
# Paste the python code from the above instruction here
# make sure you start with import arcpy

import arcpy
arcpy.management.XYTableToPoint(
    in_table=r"D:\LSU\Sp_2025\GEOG4057\assignment-6\points.csv",
    out_feature_class=r"D:\LSU\Sp_2025\GEOG4057\assignment-6\assignment-6.gdb\point_fromxy",
    x_field="X",
    y_field="Y",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)


<Result 'D:\\LSU\\Sp_2025\\GEOG4057\\assignment-6\\assignment-6.gdb\\point_fromxy'>

## Commentating the example code:(20 pnts)
 - Using InsertCursor to do the same thing but project the geometry to a different spatial reference 

- The above code only imports the csv to a feature class but not converting from a GCS to a project coordinate system
- We will use the following code to do the work and project the data to "NAD 1983 USGS Contiguous USA Albers" system with the WKID = 102039
- Look the code up from here: https://epsg.io/102039

In [2]:
import arcpy
import csv
import os

# Edit the path names for your case
csv_file_path = r"D:\LSU\Sp_2025\GEOG4057\assignment-6\points.csv"
geodatabase_path = r"D:\LSU\Sp_2025\GEOG4057\assignment-6\assignment-6.gdb"
feature_class_name = "point_albers"

# Determine the number of fields and field names from the CSV header
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)
    num_fields = len(header)

# Create SpatialReference objects: 
sr_gcs = arcpy.SpatialReference(4269) 
sr_albsers = arcpy.SpatialReference(102039)
#delete if the feature class exists

fcName = os.path.join(geodatabase_path,feature_class_name)

if arcpy.Exists(fcName):
    arcpy.management.Delete(fcName)


# Create a new point feature class

arcpy.CreateFeatureclass_management(geodatabase_path, feature_class_name, "POINT", spatial_reference=sr_albsers)
# Add fields to the feature class based on the CSV header

# first, read the exisint fields 
describe = arcpy.da.Describe(fcName)
field_names = [field.name for field in describe["fields"]]


for field in header:
    if field not in field_names:
        arcpy.AddField_management(fcName, field, "TEXT")

# Create a cursor for inserting point features with additional fields
cursor_fields = ["SHAPE@"] + header
with arcpy.da.InsertCursor(fcName, cursor_fields) as cursor:
    # Read data from the CSV file and create point features with attributes
    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        
        # Skip the header row (if present)
        next(csv_reader, None)
        
        for table_value in csv_reader:
            x = float(table_value[0]) # note that the first two fields in the csv must be longitude and latitude
            y = float(table_value[1])
            pnt = arcpy.PointGeometry(arcpy.Point(x,y),sr_gcs)
            pnt_alberse = pnt.projectAs(sr_albsers)

            row = (pnt_alberse,) + tuple(table_value)
            cursor.insertRow(row)

print(f"New point feature class '{feature_class_name}' created in '{geodatabase_path}' with {num_fields} fields.")



New point feature class 'point_albers' created in 'D:\LSU\Sp_2025\GEOG4057\assignment-6\assignment-6.gdb' with 5 fields.


**Edit this block to answer these three questions**

Q1: Line by line, explain what was done in the code. (10 pnts)

-first three lines are for the necessary modules
-then i have defined the necessary paths
-with open csv, it reads first row,and report how many fields are present by counting the number of columns in the header
-4269 and 102039 are for spatial reference objects
-fcName variable combines geodatabase_path and feature_class_name
-if arcpy.Exists checks if a feature class with same name already exists
-arcpy.CreateFeatureclass creates a new feature class in the specified geodatabase
-describe-arcpy.da.Describe to for loop- this part retrieves existing fields in newly created feature class, then iterates through the CSV file header and add as new text file.
-cursor fields works through a list of field,, arcpy.da.InsertCursor insert new rows to feature class
-for table value in csv reader loop- extract lat and long, point geometry is created using sr_gcs, sr_albers reprojected this to NAD83/Albers, then a row is created and row is inserted in feature class using cursor
-final output message is printed by print of last line

Q2: What do the cod 4269 and 102309 represent? (5 pnts)

EPSG 4269: refers to NAD83(North American Datum 1983) geographic corordinate system
EPSG 102039: refers to NAD83/ Conus Albers (Albers Equal Area projection) which is a conic projection.

Q3: Explain what is SHAPE@ (5 pnts)

The SHAPE@ allows to work with geometric properties like points,lines,polygons. SHAPE@ allows user to insert the geometry of a feature. including the coordinate values. It is very often used in insert and update cursor.


## Use arcpy.Exist() (10 pnts)

- This code checks the existence of a specified dataset within an ArcGIS workspace.
- Fill the dataset_name and workspace_path variables with the database and the feature class name (points) from the last block
- Print a message indicating whether the dataset exists or not.

In [3]:
import arcpy
import os

# Prompt for user input
dataset_name = "point_albers"
workspace_path = r"D:\LSU\Sp_2025\GEOG4057\assignment-6\assignment-6.gdb"
fullname = os.path.join(workspace_path,dataset_name)
# Check if the dataset exists
if arcpy.Exists(fullname):
    print(f"The dataset '{dataset_name}' exists in the workspace.")
else:
    print(f"The dataset '{dataset_name}' does not exist in the workspace.")

The dataset 'point_albers' exists in the workspace.


**Edit this block to answer the question**

Q4: Line by line describe what was done by the code in the block above. (10 pnts) 

first two lines are packages to import

dataset_name defines the dataset that needs to be checked

workspace path defines the workspace of gdb

os.path.join() combines workspace path and dataset name in a full path

if else statement checks if the dataset exists or not and print the message


## Use arcpy.Walk() (10 pnts)

- This code uses arcpy.Walk() to iterate through all feature datasets in the geodatabase you just used.
- Modify the name of "workspace" to make the code work
- Run the code to list all the feature classes within each dataset.


In [4]:
import arcpy

# Define the workspace
workspace = r"D:\LSU\Sp_2025\GEOG4057\assignment-6\assignment-6.gdb" # note that you need to put a full path name for this workspace, even running it in ArcGIS Pro. 

# Use arcpy.Walk() to iterate through feature datasets
for dirpath, dirnames, filenames in arcpy.da.Walk(workspace, datatype="FeatureClass"):
    for filename in filenames:
        print(f"Feature Class in {dirpath}: {filename}")

Feature Class in D:\LSU\Sp_2025\GEOG4057\assignment-6\assignment-6.gdb: point_fromxy
Feature Class in D:\LSU\Sp_2025\GEOG4057\assignment-6\assignment-6.gdb: point_albers


**Edit this block to answer the question**

Q5: line by line, describe what was done by the code in the block above. (10 pnts)

first line is to import arcpy

workspace will define the path

arcpy.da.Walk() will iterate through the directory and will retrieve the data of different types like class, table, raster etc. it will loop through each directory

if it finds anything, the last line is for printing of feature class with file name

## Use List Comprehension (10 pnts)

- The following block uses list comprehension to generate a list of .shp files in a specified folder.
- Choose the folder name from assignment 4 data/paris subfolder where many shapefiles are located and use it for folder_path
- Run the code to print the list of file names


- In the second block, the code combines the os.walk() function with the list comprehension to  list all shapefiles in a folder including subfolders



In [5]:
import os

# Specify the folder path
folder_path = r"D:\LSU\Sp_2025\GEOG4057\geog4057-main\data\ModelBuilder_data\ParisData"

# Use list comprehension to generate a list of .shp files
shp_files = [file for file in os.listdir(folder_path) if file.endswith(".shp")]

# Print the list of .shp files
print("Shapefiles in the folder:")
for shp_file in shp_files:
    print(shp_file)

Shapefiles in the folder:
Metro_Entrances.shp
Warehouses.shp


In [6]:
import os
# Specify the folder path
folder_path = r"D:\LSU\Sp_2025\GEOG4057\geog4057-main\data\ModelBuilder_data\ParisData"

print("Shapefiles in the folder:")
for root, dirs, files in os.walk(folder_path):
    shp_files = [file for file in files if file.endswith(".shp")]
    for shp_file in shp_files:
            print(shp_file)

Shapefiles in the folder:
Metro_Entrances.shp
Warehouses.shp
Stores.shp
Parks.shp
Metro_Lines.shp
Metro_LinesAOI.shp
Metro_Stations.shp
Streets.shp


**Edit this block to answer the question**

Q6: Describe what was done by the code in first blocks above. (5 pnts)

folder path is for defining the path of folder

list comprehension will filters the list to include the files that only ends with .shp extension. these files will be stored in shp_files variable

for loop is used to iterate through the shp_files variable and print each file name


Q7: Describe what was done by the code in second blocks above. (5 pnts)

os.walk() is used instead of os.listdir() here, it goes through the folder apth and the directory.
root, dirs and files are used to represent the current directory
list comprehension will filter to include files that ends with .shp

## Use arcpy.da.SearchCursor (10 pnts)

- This code uses arcpy.da.SearchCursor to extract attribute information from a feature class.
- Enter the name of the "points" feature class and the field(s) from the feature class to extract.
- Run and display the extracted data.


In [7]:
import arcpy

# Prompt for user input
fc_path = geodatabase_path + "/" + feature_class_name
fields_to_extract = ["X", "Y", "Field1", "Field2", "Field3"]

# Use arcpy.da.SearchCursor to extract data
with arcpy.da.SearchCursor(fc_path, fields_to_extract) as cursor:
    print("Extracted Data:")
    for row in cursor:
        print([row[i] for i in range(len(fields_to_extract))])

Extracted Data:
['-118.2437', '34.0522', '10', '15', '20']
['-73.935242', '40.730610', '8', '11', '14']
['-95.369804', '29.760427', '12', '18', '22']
['-121.886329', '37.338208', '6', '9', '12']
['-87.629798', '41.878113', '15', '22', '28']
['-97.743061', '30.267153', '11', '16', '21']
['-122.419416', '37.774929', '9', '13', '17']
['-118.242766', '34.052235', '14', '21', '26']
['-71.058880', '42.360081', '7', '10', '13']
['-81.694360', '41.499321', '12', '17', '22']


**Edit this block to answer the question**

Q8: Line by line, explain what was done in the code. 10 pnts

first line is to import arcpy

fc_path will combine the path to a geodatabase

fields to extract list down the fields of the attribute table that will be extracted from feature class

arcpy.da.SearchCursor creates cursor to search through each rows in feature class and will extract the fields that we have declared under the variable fields_to_extract

with statement means, the cursor will close once the block of code is executed and finised

print statement will print the extracted data

## Use addField and field Calculator (20 pnts)

- Run the following code block (with "points" feature class added to the last map before you switched into the notebook interface)

In [11]:
import arcpy
fc = "point_fromxy"
newfieldName = "all"
arcpy.AddField_management(fc, newfieldName, "DOUBLE")
expression = "sum(!Field1!,!Field2!,!Field3!)"
codeblock = """
def sum(*fields):
    sum = 0
    for field in fields:
        sum += field
    return sum
"""
arcpy.CalculateField_management(fc, newfieldName, expression, "", codeblock)

<Result 'point_fromxy'>

**Edit this block to answer the questions**

Q9: line by line, describe what was done by the code in the block above. 10 pnts

first line is to import arcpy module

fc is the variable to defined the points_fromxy that we created at starting of this assignment

then the loop is for summing up the values that will go row by row, we have field 1,2,3 and our new field is named as all. field 1,2,3 summation for each row will be added in the all field of the attribute table. The sum function will do the summation

Q10: Open the attribute table of "points" and check if the attribute table has a new filed "all" and correct values. Right-click the "all" field and click field calculator. Describe what you see in the field calculator interface. Compare the python code versus the field calculator interface. 10 pnts

Yes there is a new field named as all with correct summation.If I click to open the calculate field, the options are input table with the point_fromxy2, field name is all, expression type is python3.The Fields box has all the field listed and helpers is the mathematical oepration listed....and the 'all =' block has the sumamtion command with field1, field 2, and field 3. There is also a block named as codeblock....then there is also apply and ok button.

The difference is the outlook of the gui interface, in the python code we need to define all the things and variables in a processed manner, but the gui will give us the option to select by seeing the available options which may be easier for a new user.

## Submit the assignment ontime (10 pnts)

- Late penalty is 10 points from here